In [1]:
from sage.rings.finite_rings.hom_finite_field import FiniteFieldHomomorphism_generic

In [2]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder


from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite fiel `\GF{p^m}`
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are roots of `generating_pol
    """
    def __init__(self, defining_set, generating_pol, field):
        """
        Initialize.
        """
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = field
        self._field_L = generating_pol.base_ring()
        
        if (not self._field.is_field() or not self._field.is_finite()):
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """ 
        return self._generating_pol
    
    def get_defining_set(self):
        """
        Return the defining set
        """ 
        return self._defining_set
    
    def get_parity_pol(self):
        """
        Return the parity polynomial
        """
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def get_parity_check_matrix(self):
        """
        Return a parity check matrix of the code.
        """
        V, from_V, to_V = self._field_L.vector_space(self._field, map = True)

        parity = self.get_parity_pol()

        vector_L = []
        for i in range(len(parity)):
            vector_L = vector_L + parity[i].list()

        vector_F = []
        for i in range(len(vector_L)):
            vector_F = vector_F + to_V(vector_L[i]).list()

        matriz_F = matrix(self._length, vector_F)
        matriz_F = matriz_F.T
        
        return matriz_F

    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        H = self.get_parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        
        return G
    
    def dimension(self):
        """
        Return the dimension of the code
        """
        
        return rank(self.get_generator_matrix())
    
    def _repr_(self):
        """
        Representation of a Goppa code
        """
        return "[{}, {}] Goppa code".format(self._length, self.dimension())

class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self.code().get_generator_matrix()
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m``: a vector to encode
        """
        return m * self.get_generator_matrix()
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code``: code associated with the decoder
    """
    
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
        
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        
        INPUT:
        - ``c``: a codeword of ``self``
        """
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):    
            syndrome = syndrome + embFL(c[i])*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """
        return self._generating_pol
    
    def decode_to_code(self, word):
        r"""
        Corrects the errors in ``word`` and returns a codeword.
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        i = 1
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        secLF = embFL.section()

        # Step 1
        S = self.get_syndrome(word)
        
        if S == 0:
            return word

        # Step 2
        r_prev = self.get_generating_pol()
        t = floor(self.get_generating_pol().degree()/2)
        r_i = S
        U_prev = 0
        U_i = 1
        
        # Steps 3 and 4
        while r_i.degree() >= t:
            (q, r) = r_prev.quo_rem(r_i)
            aux_r_i = r_i
            aux_U_i = U_i
            r_i = r
            U_i = q * U_i + U_prev
            r_prev = aux_r_i
            U_prev = aux_U_i  
            i += 1
            

        # Step 5
        # make sigma monic     
        sigma = U_i/U_i.coefficients()[-1];
        eta = (-1)^i * r_i / U_i.coefficients()[-1];
        
        # roots of sigma are the locations of the errors
        roots_loc = []

        for root in sigma.roots():
            roots_loc = roots_loc + [self._defining_set.index(root[0])]
        
        error = [0] * len(self._defining_set)
        x = self.get_generating_pol().parent().gen()
        sigma_diff = sigma.diff(x)
        
        for i in range(0, len(sigma.roots())):
            error[roots_loc[i]] = secLF(eta.subs(x=sigma.roots()[i][0])/(sigma_diff.subs(x=sigma.roots()[i][0])))
        
        x = word - vector(self.code()._field, error)
        
        return x

Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

# Niederreiter cryptosystem

In [3]:
def get_weight(c):
    w = 0
    
    for elem in c:
        if elem != 0:
            w += 1
    
    return w

In [4]:
class Niederreiter:
    r"""
    Implementation of Niederreiter cryptosystem.
    
    INPUT:
    - ``g`` -- a monic polynomial with coefficients in
    a finite fiel `\GF{p^q}`
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are roots of `generating_pol
    """
    def __init__(self, n, p, q, g):
        """
        Initialize.
        """
        
        if not g.is_irreducible:
            raise ValueError("ERROR. Generating polynomial isn't irreducible")
        
        F = GF(2) # todo esto es asi?
        L = GF(2^q)
        
        # defining set
        defining_set = []
        while len(defining_set) != n:
            aux = L.random_element()
            if g(aux) != 0 and aux not in defining_set:
                defining_set = defining_set + [aux]
    
        C = Goppa(defining_set, g, F)
        k = C.get_parity_matrix().nrows()
        n = C.get_parity_matrix().ncols()
        
        # random binary nonsingular matrix
        S = matrix(F, k, [choice(F.list()) for i in range(k^2)])
        
        while rank(S) < k:
            i = randint(0, k-1)
            j = randint(0, k-1)
            S[i,j] = choice(F.list())
        
        # random permutation matrix
        columns = list(range(n))
        P = matrix(F, n)
        
        for i in range(n):
            l = randint(0, len(columns)-1)
            j = columns[l]
            P[i,j] = 1
            columns.remove(j)
        
        self._goppa = C
        self._H = C.get_parity_check_matrix()
        self._t = g.degree()
        self._k = k
        self._n = n
        self._S = S
        self._P = P
        self._public_key = S * self._H * P
        
    def get_S(self):
        # TODO borrar
        return self._S
    
    def get_P(self):
        # TODO borrar
        return self._P
    
    def get_public_key(self):
        return self._public_key

    def encrypt(self, m):
        """
        Return a chipertext
        
        INPUT:
        - ``m``: a plaintext to encrypt
        """
        # codeword
        c = m * self._public_key.T
        
        return c
    
    def decrypt(self, c):
        word = self._S^(-1) * c.column()
        z = (self._H^(-1) * word).column()
        # TODO:
        """D = GoppaDecoder(self._goppa)
        word = D.decode_to_code(z)
        
        message = (self._S).solve_left(word)
        """
        return 0

In [5]:
p = 1
q = 6
n = 2^q
print(n)
t = floor((2+(2^q-1)/q)/2)

F = GF(2^p)
L = GF(2^q)
a = L.gen()
b = F.gen()
R.<x> = L[]

while True:
    g = R.random_element(t)
    
    if g.is_irreducible() and g.is_monic():
        break

print(g)

N = Niederreiter(n, p, q, g)

64
x^6 + (z6^4 + z6 + 1)*x^5 + (z6^5 + z6^3 + z6^2 + z6 + 1)*x^4 + z6^4*x^3 + z6^2*x^2 + (z6^5 + z6^4 + z6^3 + z6^2 + z6)*x + z6^4 + z6^3 + z6


In [6]:
public_key = N.get_public_key()
show(public_key)

message = []
while len(message) != public_key.nrows():
    message = message + [choice(F.list())]

message = vector(F, message)

print("\nMessage to encrypt:")
print(message)

encrypted_message = N.encrypt(message)

print("\nEncrypted message:")
print(encrypted_message)
print(len(encrypted_message))

28 x 64 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)


Message to encrypt:
(0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1)

Encrypted message:
(0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1)
64


In [7]:
message2 = N.decrypt(encrypted_message)

ValueError: z6^4 + z6^3 + 1 is not in the image of Ring morphism:
  From: Finite Field of size 2
  To:   Finite Field in z6 of size 2^6
  Defn: 1 |--> 1

# Temporal